# Entrenamiento del modelo.

Para la fase 1 del proyecto sustituto del curso Modelos y Simulaciones I, 202402, Universidad de Antioquia, se ha tomado como objeto de estudio la competencia de Kaggle [New York City Taxi Trip Duration](https://www.kaggle.com/competitions/nyc-taxi-trip-duration/), utilizando como guía el notebook [Beginner Friendly Approach](https://www.kaggle.com/code/jaysabnis/new-york-trip-duration-beginner-friendly-approach/notebook) del usuario Jay Sabnis.

En este notebook se encuentra el proceso de entrenameinto del modelo. Para el modelo se utilizara el algoritmo CatBoostRegressor, el cual preseto mejores resultados en comparación con otros 5 algorimos que fueron probados en el notebook de Jay Sabnis. Utilizando la libreria **optuna** se realizo la busqueda un conjunto de hiperparametros que mejoraran los resultados del modelo, los cuales son usado para el entrenamiento del modelo en este recurso.

In [1]:
# Se clona el repositorio de github en el entrno del notebook.
# Se descomprimer el zip que contiene los datos para realizar el entrenamiento.

!git clone https://github.com/franco-arroyave/Modelos-y-Simulaciones-I
!unzip -uq "/content/Modelos-y-Simulaciones-I/data/data_clean.zip" -d "/content/"
!cp "/content/Modelos-y-Simulaciones-I/model/model.cbm" "/content/"

fatal: destination path 'Modelos-y-Simulaciones-I' already exists and is not an empty directory.


In [2]:
# Instalación e importación de las librerias necesarias.

!pip install catboost

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as r2
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Función utilizada para evaluar el modelo.

def rmsle(y_true, y_pred):
    """
    Calculate Root Mean Squared Logarithmic Error between two arrays.

    Parameters:
        y_true (array-like): Ground truth values.
        y_pred (array-like): Predicted values.

    Returns:
        float: Root Mean Squared Logarithmic Error.
    """
    log1 = np.log1p(y_pred)
    log2 = np.log1p(y_true)
    squared_log_errors = (log1 - log2) ** 2
    mean_squared_log_error = np.mean(squared_log_errors)
    return np.sqrt(mean_squared_log_error)

In [4]:
# Carga de los datos preprocesados.
# Los datos se encuentra almacenados en el repositorio en la carpeta data.

train_data=pd.read_csv("data_clean.csv")

In [5]:
# Separación de los datos entre variables predictoras y predichas.
# Separación de los datos entre entrenamiento (80%) y test (20%)

X=train_data.drop(["trip_duration"],axis=1)
y=train_data["trip_duration"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [6]:
# Hiperparametros obtenidos por la libreria obtuna.

best_param={'learning_rate': 0.097249166106368,
             'depth': 10,
             'l2_leaf_reg': 0.9191504373151137,
             'iterations': 1000}

In [7]:
# Etrenamiento del modelo.
# Nota: El entrenamiento del modelo puede tomar al rededor de 5 minutos,
# para deducir los tiempos de ejecuccion el modelo preentrenado se encuentra
# almacenado en el repositorio, este puede ser cargado o vuelto a entrenar

model=CatBoostRegressor(**best_param,verbose=0)

#model.fit(X,y)

model.load_model("model.cbm")

In [8]:
# Prueba del modelo con datos test

y_pred=model.predict(X_test)

print("RMSLE: ",round(rmsle(y_test, y_pred),4))
print("R2: ",round(r2(y_test, y_pred),4))

RMSLE:  0.436
R2:  0.7271


In [9]:
# Prueba del modelo con datos train

y_pred=model.predict(X_train)

print("RMSLE: ",round(rmsle(y_train, y_pred),4))
print("R2: ",round(r2(y_train, y_pred),4))

RMSLE:  0.436
R2:  0.7277
